In [126]:
import pandas as pd
import numpy as np
import re
import os
import json

In [127]:
# strip the alphabets
def strip(data):
    pattern = re.compile('[^0-9]', re.UNICODE)
    return re.sub(pattern, "", str(data))

In [128]:
filePath = '/Users/helix/Coding/attitude/data/'

In [129]:
data = pd.read_excel(filePath+'all_people.xlsx')
data = data[['id', '政治态度得分']]
data.columns = ['id', 'score']
data.dropna()
data.id = data.id.apply(strip)

In [130]:
data.head()

,id,score
0,201922301,4.14
1,20191721032,3.36
2,201906005,3.47
3,201906004,3.54
4,201823164,3.80


In [131]:
filePath = '/Users/helix/Coding/attitude/data/all_json/'
fileNames = os.listdir(filePath)
json_file = []
for fileName in fileNames:
    File = open(filePath+fileName)
    test = json.load(File)
    #print(fileName)
    test = np.array(test['vectors'])
    json_file.append([strip(fileName), test])
    File.close()

In [132]:
json_file = pd.DataFrame(json_file, columns=['id', 'vector'])

In [133]:
json_file.head()

,id,vector
0,1901140015,"[[-1.579109286842105, 2.9206601189473687, -0.1..."
1,1901362601,"[[-1.7639175999999999, -3.2602275249999995, 2...."
2,18110336,"[[-4.3450105425, 0.24810093499999986, 4.423870..."
3,190901191133,"[[-5.1014285, 2.597416484444445, 5.91408673333..."
4,19041230,"[[-3.6661185405555554, 0.782676534861111, 2.68..."


In [134]:
data.head()

,id,score
0,201922301,4.14
1,20191721032,3.36
2,201906005,3.47
3,201906004,3.54
4,201823164,3.80


In [135]:
data = pd.merge(data, json_file)

In [136]:
data = data.dropna()

In [137]:
data.head()

,id,score,vector
0,201922301,4.14,"[[-4.648576633333334, -0.1527824666666667, 5.5..."
1,20191721032,3.36,"[[-2.462165133333333, -1.2517511333333333, 6.6..."
2,201906004,3.54,"[[-3.7282987317142857, 1.0043741571428575, 5.0..."
3,201822048,3.98,"[[-2.4070685, 1.4974684000000003, 0.9653421999..."
4,201822042,3.51,"[[-5.945544979999999, 2.07371892, 5.2692115799..."


In [138]:
data.shape

(3423, 3)

In [139]:
def truncate(content):
    return content[0]

In [151]:
data['vector_100'] = data.vector.apply(truncate)

In [152]:
data.head()

,id,score,vector,vector_100
0,201922301,4.14,"[[-4.648576633333334, -0.1527824666666667, 5.5...","[-4.648576633333334, -0.1527824666666667, 5.57..."
1,20191721032,3.36,"[[-2.462165133333333, -1.2517511333333333, 6.6...","[-2.462165133333333, -1.2517511333333333, 6.63..."
2,201906004,3.54,"[[-3.7282987317142857, 1.0043741571428575, 5.0...","[-3.7282987317142857, 1.0043741571428575, 5.03..."
3,201822048,3.98,"[[-2.4070685, 1.4974684000000003, 0.9653421999...","[-2.4070685, 1.4974684000000003, 0.96534219999..."
4,201822042,3.51,"[[-5.945544979999999, 2.07371892, 5.2692115799...","[-5.945544979999999, 2.07371892, 5.26921157999..."


In [153]:
data.score = data.score.astype(np.float32)

In [154]:
target = data.score.to_list()

In [155]:
target =  np.array(target)
target = target.astype(np.float32)

In [156]:
vector_100 = data.vector_100.to_list()

In [157]:
vector_100 = np.array(vector_100)

In [158]:
from tensorflow.python.keras.models import Sequential
import tensorflow as tf
from tensorflow import keras
from tensorflow import initializers as init

In [159]:
model = Sequential()
model.add(keras.layers.Dense(4, input_shape=(100,), activation='sigmoid'))
model.add(keras.layers.Dense(1))

In [160]:
model.compile(loss = 'mse',
             optimizer='adam')  

In [161]:
model.fit(vector_100/10, target, epochs=30, validation_split=0.1)

Epoch 1/30
97/97 [==============================] - 0s 2ms/step - loss: 13.9708 - val_loss: 10.1830
Epoch 2/30
97/97 [==============================] - 0s 3ms/step - loss: 9.8577 - val_loss: 7.4920
Epoch 3/30
97/97 [==============================] - 0s 2ms/step - loss: 7.4459 - val_loss: 5.3466
Epoch 4/30
97/97 [==============================] - 0s 3ms/step - loss: 5.1433 - val_loss: 3.4398
Epoch 5/30
97/97 [==============================] - 0s 2ms/step - loss: 3.4756 - val_loss: 2.2319
Epoch 6/30
97/97 [==============================] - 0s 3ms/step - loss: 2.3762 - val_loss: 1.4341
Epoch 7/30
97/97 [==============================] - 0s 2ms/step - loss: 1.6233 - val_loss: 0.9072
Epoch 8/30
97/97 [==============================] - 0s 2ms/step - loss: 1.1057 - val_loss: 0.5640
Epoch 9/30
97/97 [==============================] - 0s 3ms/step - loss: 0.7557 - val_loss: 0.3520
Epoch 10/30
97/97 [==============================] - 0s 3ms/step - loss: 0.5250 - val_loss: 0.2275
Epoch 11/30
97/97

In [169]:
num_sen = []
for i in data.vector:
    num_sen.append(i.shape[0])

In [174]:
num_sen = np.array(num_sen)

In [176]:
np.mean(num_sen)

70.13701431492842

In [262]:
max_sen = np.mean(num_sen) + 2 * np.std(num_sen)
max_sen = int(max_sen)
max_sen# = 50

168

In [263]:
# 句子的个数可以覆盖的范围
np.sum(num_sen < max_sen) / len(num_sen)

0.9932807478819748

In [264]:
def uniform(content):
    if (content.shape[0]>=max_sen):
        return content[0:max_sen]
    else:
        return np.pad(content, [[0, max_sen-content.shape[0]], [0, 0]])

In [265]:
data.vector = data.vector.apply(uniform)

In [266]:
vector = np.array(data.vector.to_list())

In [267]:
vector.shape

(3423, 168, 100)

In [278]:
model2 = Sequential()
model2.add(keras.layers.Dense(20, input_shape=(168,100), activation='relu'))
model2.add(keras.layers.Dense(10))
model2.add(keras.layers.Dense(5, activation='relu'))
model2.add(keras.layers.Dense(1, activation='linear'))

In [279]:
model2.compile(loss='mse',
                optimizer='rmsprop')

In [281]:
model2.fit(vector, target, epochs=50, validation_split=0.1, batch_size=128)

Epoch 1/50
25/25 [==============================] - 1s 23ms/step - loss: 0.1624 - val_loss: 0.2007
Epoch 2/50
25/25 [==============================] - 1s 23ms/step - loss: 0.1624 - val_loss: 0.2050
Epoch 3/50
25/25 [==============================] - 1s 24ms/step - loss: 0.1629 - val_loss: 0.1732
Epoch 4/50
25/25 [==============================] - 1s 22ms/step - loss: 0.1624 - val_loss: 0.2305
Epoch 5/50
25/25 [==============================] - 1s 25ms/step - loss: 0.1629 - val_loss: 0.2149
Epoch 6/50
25/25 [==============================] - 1s 23ms/step - loss: 0.1629 - val_loss: 0.1403
Epoch 7/50
25/25 [==============================] - 1s 25ms/step - loss: 0.1628 - val_loss: 0.1584
Epoch 8/50
25/25 [==============================] - 1s 24ms/step - loss: 0.1620 - val_loss: 0.1325
Epoch 9/50
25/25 [==============================] - 1s 23ms/step - loss: 0.1630 - val_loss: 0.1354
Epoch 10/50
25/25 [==============================] - 1s 23ms/step - loss: 0.1635 - val_loss: 0.1523
Epoch 11/

KeyboardInterrupt: 